In [ ]:
import urllib.request
from urllib.request import urlopen
import requests
import folium
import json
import geojson
import pandas as pd
import plotly.express as px
from datetime import datetime

In [ ]:
#0501013B0 = Amoxicillin

url_0501013B0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501013B0&format=json"
response_0501013B0 = urllib.request.urlopen(url_0501013B0)
data_0501013B0 = json.loads(response_0501013B0.read())
flat_data_0501013B0 = pd.json_normalize(data_0501013B0)
flat_data_0501013B0 = flat_data_0501013B0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501013B0 = flat_data_0501013B0.drop(columns=['items', 'quantity'])
flat_data_0501013B0 .rename(columns={'actual_cost': 'Amoxicillin'}, inplace=True)
flat_data_0501013B0

In [ ]:
#0501030I0 = Doxycycline Hyclate

url_0501030I0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501030I0&format=json"
response_0501030I0 = urllib.request.urlopen(url_0501030I0)
data_0501030I0 = json.loads(response_0501030I0.read())
flat_data_0501030I0 = pd.json_normalize(data_0501030I0)
flat_data_0501030I0 = flat_data_0501030I0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501030I0 = flat_data_0501030I0.drop(columns=['items', 'quantity'])
flat_data_0501030I0.rename(columns={'actual_cost': 'Doxycycline Hyclate'}, inplace=True)
flat_data_0501030I0

In [ ]:
#0501021L0 = Cefalexin

url_0501021L0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501021L0&format=json"
response_0501021L0 = urllib.request.urlopen(url_0501021L0)
data_0501021L0 = json.loads(response_0501021L0.read())
flat_data_0501021L0= pd.json_normalize(data_0501021L0)
flat_data_0501021L0 = flat_data_0501021L0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501021L0 = flat_data_0501021L0.drop(columns=['items', 'quantity'])
flat_data_0501021L0.rename(columns={'actual_cost': 'Cefalexin'}, inplace=True)
flat_data_0501021L0

In [ ]:
join_1 = flat_data_0501013B0.join(flat_data_0501030I0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged = join_1.join(flat_data_0501021L0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged.fillna(0, inplace=True)
all_antibiotics_merged['Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)']= all_antibiotics_merged.iloc[:, -3:].sum(axis=1)
all_antibiotics = all_antibiotics_merged.reset_index()
all_antibiotics.rename(columns={'row_name': 'Clinical Commissioning Group (CCG)', 'row_id': 'CCG code', 'date': 'Date'}, inplace=True)
all_antibiotics

In [ ]:
all_antibiotics_plot = all_antibiotics.groupby(['Date']).sum()
all_antibiotics_plot = all_antibiotics_plot.reset_index()
all_antibiotics_plot = all_antibiotics_plot.round(1)

In [ ]:
pd.options.plotting.backend = "plotly"
fig = px.bar(all_antibiotics_plot, x='Date', y= ["Amoxicillin", "Doxycycline Hyclate", 'Cefalexin'],
color_discrete_sequence=["#003087", "#0072CE", "#41B6E6"],
labels={"value": "Cost (£)", "variable": "Antibiotic"},  
title= "Total cost of Amoxicillin, Doxycycline Hyclate, and Cefalexin (£) per month")
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig.show()

In [ ]:
#CCG pop from NHS digital 
csv_url = "https://files.digital.nhs.uk/40/2232E5/gp-reg-pat-prac-all.csv"
req = requests.get(csv_url)
url_content = req.content
csv_file = open('ccg_pop.csv', 'wb')
csv_file.write(url_content)
csv_file.close()
df1 = pd.read_csv('ccg_pop.csv')
CCG_pop = df1.groupby(['CCG_CODE']).sum().reset_index()
CCG_pop.rename(columns={'CCG_CODE': 'CCG code', 'NUMBER_OF_PATIENTS': 'Number of patients registered at GP practices'}, inplace=True)
CCG_pop

In [ ]:
current_year = datetime.now().year
current_year_str = str(current_year)

In [24]:
all_antibiotics["Date"] = pd.to_datetime(all_antibiotics["Date"]).apply(lambda x: x.strftime("%Y"))
all_antibiotics_current_year = all_antibiotics.loc[all_antibiotics['Date'] == current_year_str]
df1 = all_antibiotics_current_year.groupby(["CCG code", "Clinical Commissioning Group (CCG)"]).sum()
df2 = df1.drop(columns=['Amoxicillin', 'Doxycycline Hyclate', 'Cefalexin'])
df3 = df2.reset_index()
df4 = df3.join(CCG_pop, rsuffix='CCG code')
df5 = df4.drop(columns=['CCG codeCCG code'])
df5["Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients"] = df5["Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)"]/(df5["Number of patients registered at GP practices"]/1000)
df5.round(2)
df6 = df5.sort_values(by= "Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients" , ascending=True)
df6

,CCG code,Clinical Commissioning Group (CCG),"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)",Number of patients registered at GP practices,"Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients"
57,09D,NHS BRIGHTON AND HOVE CCG,21748.69,324743,66.972006
58,10Q,NHS OXFORDSHIRE CCG,56942.15,781104,72.899575
90,93C,NHS NORTH CENTRAL LONDON CCG,124503.81,1704940,73.025332
104,W2U3Z,NHS NORTH WEST LONDON CCG,198588.36,2679014,74.127407
69,15C,"NHS BRISTOL, NORTH SOMERSET AND SOUTH GLOUCEST...",77728.43,1044172,74.440255
...,...,...,...,...,...
15,01J,NHS KNOWSLEY CCG,23941.40,168955,141.702820
86,84H,NHS COUNTY DURHAM CCG,79601.84,558062,142.639778
4,00R,NHS BLACKPOOL CCG,25123.75,175261,143.350489
64,12F,NHS WIRRAL CCG,48963.45,338526,144.637192


In [26]:
with urlopen('https://openprescribing.net/api/1.0/org_location/?org_type=ccg') as response:
    data_ccg_geojson = json.load(response)

m = folium.Map(
    location=[53, 0.13],
    tiles="cartodbpositron",
    zoom_start=6.3,
)

folium.Choropleth(
    geo_data=data_ccg_geojson,
    name="choropleth",
    data=df6,
    columns=["CCG code", "Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients"],
    key_on="feature.properties.code",
    fill_color="YlGn",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="Cost (£) per 1000 GP registered patients",
).add_to(m)

folium.LayerControl().add_to(m)

m